# FluidityNonlocal: Stress Relaxation with Spatial Fluidity Diffusion

## Learning Objectives

1. **Understand spatial relaxation dynamics**: How fluidity diffusion homogenizes stress across the gap during relaxation
2. **NLSQ fitting**: Fit relaxation modulus G(t) with spatial coupling
3. **Bayesian inference**: Quantify parameter uncertainty (D_f, G, eta_s, lambda_0, tau_eq, a, c)
4. **Fluidity homogenization**: Visualize spatial profile evolution from initial heterogeneity to uniform state
5. **Model diagnostics**: ArviZ convergence checks (R-hat, ESS, trace plots)

---

## Theoretical Background

### Relaxation Protocol

Sudden strain imposition γ₀ at t=0, then zero strain rate:

$$
\gamma(t) = \gamma_0 H(t), \quad \dot{\gamma}(t) = \gamma_0 \delta(t)
$$

where H(t) is the Heaviside step function.

### Governing Equations

**Viscoelastic stress** (Maxwell backbone):
$$
\sigma(y,t) + \lambda(y,t) \frac{\partial \sigma}{\partial t} = \eta_s \dot{\gamma}(t)
$$

**Fluidity evolution** (diffusion + thixotropy):
$$
\frac{\partial f}{\partial t} = \frac{1 - f}{\tau_{\text{eq}}} + a f |\dot{\gamma}|^c + D_f \frac{\partial^2 f}{\partial y^2}
$$

where λ(y,t) = 1/f(y,t) is the relaxation time.

### Relaxation Modulus

$$
G(t) = \frac{\langle \sigma(y,t) \rangle}{\gamma_0}
$$

Initial condition: σ(y,0) = G·γ₀ (instantaneous elastic response), f(y,0) can be spatially heterogeneous from prior shear history.

### Key Physics

1. **Elastic jump**: G(0⁺) = G (shear modulus)
2. **Spatial homogenization**: D_f diffuses fluidity from high-f (fluid) to low-f (solid) regions
3. **Structural recovery**: f → 1 (equilibrium) via 1/τ_eq aging
4. **Decay timescale**: Controlled by λ_avg(t) = 1/⟨f(y,t)⟩ and τ_eq

---

## Setup

In [ ]:
# Google Colab setup (installs RheoJAX if not present)
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
    
    # Install rheojax
    !pip install -q rheojax
    
    # Create output directory
    !mkdir -p outputs/fluidity/nonlocal/relaxation
    
except ImportError:
    IN_COLAB = False
    print("Running locally")
    
    # Create output directory (local)
    import os
    os.makedirs("../outputs/fluidity/nonlocal/relaxation", exist_ok=True)

In [ ]:
# Float64 enforcement (CRITICAL for numerical stability)
from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# RheoJAX imports
from rheojax.models.fluidity import FluidityNonlocal
from rheojax.core.data import RheoData
from rheojax.logging import configure_logging, get_logger

# Bayesian imports
import arviz as az

# Configure logging
configure_logging(level="INFO")
logger = get_logger(__name__)

# Plotting aesthetics
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'legend.fontsize': 10,
    'figure.dpi': 100
})

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")
print(f"Float64 enabled: {jax.config.jax_enable_x64}")

---

## 1. Load Calibrated Parameters or Use Defaults

We'll use realistic parameters for a yield stress fluid with spatial heterogeneity.

In [ ]:
# Default parameters (representative of carbopol gel)
params_default = {
    'G': 100.0,           # Pa - Elastic modulus
    'eta_s': 10.0,        # Pa·s - Solvent viscosity
    'lambda_0': 1.0,      # s - Reference relaxation time (1/f_eq)
    'tau_eq': 10.0,       # s - Structural recovery time
    'a': 1.0,             # Dimensionless - Rejuvenation strength
    'c': 1.0,             # Dimensionless - Shear-rate exponent
    'D_f': 1e-4,          # m²/s - Fluidity diffusion coefficient
    'gap_width': 1e-3     # m - Gap size (1 mm)
}

# Try loading from startup simulation if available
output_dir_local = Path("../outputs/fluidity/nonlocal/relaxation")
output_dir_colab = Path("outputs/fluidity/nonlocal/relaxation")
output_dir = output_dir_colab if IN_COLAB else output_dir_local

params_file = output_dir.parent / "startup" / "fitted_params.npz"

if params_file.exists():
    logger.info(f"Loading parameters from {params_file}")
    loaded = np.load(params_file)
    params = {k: float(loaded[k]) for k in loaded.files}
    print("Loaded calibrated parameters from startup simulation")
else:
    logger.info("Using default parameters")
    params = params_default.copy()
    print("Using default parameters (no calibrated file found)")

# Display parameters
print("\nModel Parameters:")
for key, val in params.items():
    print(f"  {key:12s} = {val:.4e}")

---

## 2. Generate Synthetic Relaxation Data

Simulate stress relaxation from an initial heterogeneous fluidity profile.

In [ ]:
# Initialize model with parameters
model_true = FluidityNonlocal(
    n_points=51,
    gap_width=params['gap_width'],
    coupling='full'
)

# Set parameters
model_true.parameters.set_values({
    'G': params['G'],
    'eta_s': params['eta_s'],
    'lambda_0': params['lambda_0'],
    'tau_eq': params['tau_eq'],
    'a': params['a'],
    'c': params['c'],
    'D_f': params['D_f']
})

# Relaxation protocol parameters
gamma_0 = 0.1        # Applied strain (10%)
t_end = 100.0        # s - Total relaxation time
n_times = 200        # Time points

# Initial fluidity profile (heterogeneous from prior shear)
# Example: gradient from f=0.5 (center) to f=1.5 (walls)
y_grid = np.linspace(0, params['gap_width'], model_true.n_points)
f_init = 1.0 + 0.5 * np.cos(np.pi * y_grid / params['gap_width'])  # Cosine profile

print(f"Generating relaxation data with γ₀={gamma_0:.2%}, t_end={t_end}s")
print(f"Initial fluidity range: [{f_init.min():.3f}, {f_init.max():.3f}]")

# Simulate relaxation
result = model_true.simulate_relaxation(
    gamma_0=gamma_0,
    t_end=t_end,
    n_times=n_times,
    f_init=f_init
)

t_relax = result['time']
G_t_true = result['G_t']  # Relaxation modulus
f_profile = result['fluidity_profile']  # (n_times, n_points)

# Add 3% Gaussian noise
rng = np.random.RandomState(42)
noise_level = 0.03
noise = rng.normal(0, noise_level * np.std(G_t_true), size=G_t_true.shape)
G_t_noisy = G_t_true + noise

print(f"Generated {len(t_relax)} time points from {t_relax[0]:.2e}s to {t_relax[-1]:.2e}s")
print(f"G(0⁺) = {G_t_noisy[0]:.2f} Pa (expected: {params['G']:.2f} Pa)")
print(f"G(t_end) = {G_t_noisy[-1]:.2f} Pa")

In [ ]:
# Visualize synthetic data
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Relaxation modulus
ax = axes[0]
ax.plot(t_relax, G_t_true, 'b-', linewidth=2, label='True G(t)')
ax.plot(t_relax, G_t_noisy, 'ro', markersize=4, alpha=0.5, label='Noisy data (3%)')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Relaxation Modulus G(t) (Pa)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Stress Relaxation Data')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Fluidity profile evolution
ax = axes[1]
y_mm = y_grid * 1e3  # Convert to mm

# Plot profiles at different times
time_indices = [0, len(t_relax)//4, len(t_relax)//2, 3*len(t_relax)//4, -1]
colors = plt.cm.viridis(np.linspace(0, 1, len(time_indices)))

for i, idx in enumerate(time_indices):
    ax.plot(y_mm, f_profile[idx, :], color=colors[i], 
            linewidth=2, label=f't={t_relax[idx]:.1f}s')

ax.set_xlabel('Position y (mm)')
ax.set_ylabel('Fluidity f(y)')
ax.set_title('Fluidity Profile Homogenization')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'synthetic_relaxation_data.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nFluidity homogenization:")
print(f"  t=0s:     Δf = {f_profile[0, :].max() - f_profile[0, :].min():.4f}")
print(f"  t={t_relax[-1]:.1f}s: Δf = {f_profile[-1, :].max() - f_profile[-1, :].min():.4f}")

---

## 3. NLSQ Fitting with `test_mode='relaxation'`

Fit the relaxation modulus to estimate parameters.

In [ ]:
# Create RheoData object
rheo_data = RheoData(
    x=t_relax,
    y=G_t_noisy,
    x_label='time',
    y_label='G_t',
    test_mode='relaxation'
)

# Initialize model for fitting
model_fit = FluidityNonlocal(
    n_points=51,
    gap_width=params['gap_width'],
    coupling='full'
)

# Set initial guesses (perturbed from true values)
initial_guess = {
    'G': params['G'] * 0.8,
    'eta_s': params['eta_s'] * 1.2,
    'lambda_0': params['lambda_0'] * 0.9,
    'tau_eq': params['tau_eq'] * 1.1,
    'a': params['a'] * 0.85,
    'c': params['c'] * 1.05,
    'D_f': params['D_f'] * 1.3
}

model_fit.parameters.set_values(initial_guess)

print("Starting NLSQ optimization...")
print(f"Initial guess (perturbed from true):")
for key, val in initial_guess.items():
    true_val = params[key]
    print(f"  {key:12s} = {val:.4e} (true: {true_val:.4e}, error: {100*(val-true_val)/true_val:+.1f}%)")

# Fit with NLSQ
result_nlsq = model_fit.fit(
    rheo_data,
    gamma_0=gamma_0,
    f_init=f_init,  # Use same initial condition
    max_iter=2000,
    verbose=True
, method='scipy')

print(f"\nNLSQ Optimization complete:")
print(f"  R² = {result_nlsq.r_squared:.6f}")
print(f"  Iterations: {result_nlsq.n_iter}")
print(f"  Success: {result_nlsq.success}")

In [ ]:
# Compare fitted vs true parameters
fitted_params = model_fit.parameters.get_values()

print("\nParameter Recovery:")
print(f"{'Parameter':<12s} {'True':>12s} {'Fitted':>12s} {'Error':>10s}")
print("-" * 50)
for key in ['G', 'eta_s', 'lambda_0', 'tau_eq', 'a', 'c', 'D_f']:
    true_val = params[key]
    fitted_val = fitted_params[key]
    error = 100 * (fitted_val - true_val) / true_val
    print(f"{key:<12s} {true_val:>12.4e} {fitted_val:>12.4e} {error:>9.2f}%")

In [ ]:
# Visualize NLSQ fit
G_t_pred = model_fit.predict(rheo_data, gamma_0=gamma_0, f_init=f_init)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Fitted curve
ax = axes[0]
ax.plot(t_relax, G_t_noisy, 'bo', markersize=5, alpha=0.5, label='Data')
ax.plot(t_relax, G_t_true, 'g--', linewidth=2, label='True model')
ax.plot(t_relax, G_t_pred, 'r-', linewidth=2, label=f'NLSQ fit (R²={result_nlsq.r_squared:.4f})')
ax.set_xlabel('Time (s)')
ax.set_ylabel('G(t) (Pa)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('NLSQ Fit: Relaxation Modulus')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Residuals
ax = axes[1]
residuals = G_t_noisy - G_t_pred
ax.plot(t_relax, residuals, 'ko', markersize=4, alpha=0.6)
ax.axhline(0, color='r', linestyle='--', linewidth=1)
ax.fill_between(t_relax, -2*np.std(residuals), 2*np.std(residuals), 
                 color='gray', alpha=0.2, label='±2σ')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Residuals (Pa)')
ax.set_xscale('log')
ax.set_title(f'Residuals (σ={np.std(residuals):.3f} Pa)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'nlsq_fit_relaxation.png', dpi=300, bbox_inches='tight')
plt.show()

---

## 4. Bayesian Inference with NumPyro

Quantify parameter uncertainties using NUTS sampling with NLSQ warm-start.

In [ ]:
# Set up priors (informative based on NLSQ fit)
fitted_vals = model_fit.parameters.get_values()

# Use 20% coefficient of variation for priors
cv = 0.2
for param_name in ['G', 'eta_s', 'lambda_0', 'tau_eq', 'a', 'c', 'D_f']:
    param = model_fit.parameters.get(param_name)
    fitted_val = fitted_vals[param_name]
    
    # Set prior mean to fitted value
    # Set prior std to 20% of fitted value
    param.prior_type = 'normal'
    param.prior_params = {
        'loc': fitted_val,
        'scale': cv * fitted_val
    }

print("Priors set based on NLSQ fit (Normal with 20% CV):")
for param_name in ['G', 'eta_s', 'lambda_0', 'tau_eq', 'a', 'c', 'D_f']:
    param = model_fit.parameters.get(param_name)
    print(f"  {param_name:12s}: N({param.prior_params['loc']:.4e}, {param.prior_params['scale']:.4e})")

# Run Bayesian inference
print("\nStarting NUTS sampling (4 chains, warm-start from NLSQ)...")
result_bayes = model_fit.fit_bayesian(
    rheo_data,
    num_warmup=1000,
    num_samples=2000,
    num_chains=4,
    seed=42,
    gamma_0=gamma_0,
    f_init=f_init
)

print("\nBayesian inference complete!")

---

## 5. ArviZ Diagnostics

Convergence checks: R-hat, ESS, trace plots, pair plots.

In [ ]:
# Convert to ArviZ InferenceData
idata = az.from_numpyro(result_bayes.mcmc)

# Summary statistics
summary = az.summary(idata, hdi_prob=0.95)
print("\nPosterior Summary (95% HDI):")
print(summary)

# Check convergence
print("\nConvergence Diagnostics:")
print(f"  Max R-hat: {summary['r_hat'].max():.4f} (target: <1.01)")
print(f"  Min ESS bulk: {summary['ess_bulk'].min():.0f} (target: >400)")
print(f"  Min ESS tail: {summary['ess_tail'].min():.0f} (target: >400)")

if summary['r_hat'].max() > 1.01:
    print("  WARNING: R-hat > 1.01 detected. Consider increasing num_warmup/num_samples.")
else:
    print("  ✓ All R-hat values < 1.01 (good convergence)")

if summary['ess_bulk'].min() < 400:
    print("  WARNING: Low ESS detected. Consider increasing num_samples.")
else:
    print("  ✓ All ESS values > 400 (sufficient samples)")

In [ ]:
# Trace plots
fig = az.plot_trace(
    idata,
    var_names=['G', 'eta_s', 'lambda_0', 'tau_eq', 'a', 'c', 'D_f'],
    compact=True,
    figsize=(14, 10)
)
plt.suptitle('NUTS Trace Plots', y=1.001, fontsize=14)
plt.tight_layout()
plt.savefig(output_dir / 'trace_plots.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Pair plot (correlations)
fig = az.plot_pair(
    idata,
    var_names=['G', 'eta_s', 'lambda_0', 'tau_eq', 'D_f'],
    kind='hexbin',
    marginals=True,
    figsize=(12, 12)
)
plt.suptitle('Posterior Correlations', y=1.001, fontsize=14)
plt.tight_layout()
plt.savefig(output_dir / 'pair_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Forest plot (credible intervals)
fig = az.plot_forest(
    idata,
    var_names=['G', 'eta_s', 'lambda_0', 'tau_eq', 'a', 'c', 'D_f'],
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 6)
)
plt.title('95% Credible Intervals', fontsize=14)
plt.tight_layout()
plt.savefig(output_dir / 'forest_plot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Posterior predictive check
posterior_samples = result_bayes.posterior_samples

# Draw 100 posterior samples
n_posterior_draws = 100
indices = np.random.choice(len(posterior_samples['G']), size=n_posterior_draws, replace=False)

fig, ax = plt.subplots(figsize=(10, 6))

# Plot posterior predictions
for idx in indices:
    model_fit.parameters.set_values({
        'G': float(posterior_samples['G'][idx]),
        'eta_s': float(posterior_samples['eta_s'][idx]),
        'lambda_0': float(posterior_samples['lambda_0'][idx]),
        'tau_eq': float(posterior_samples['tau_eq'][idx]),
        'a': float(posterior_samples['a'][idx]),
        'c': float(posterior_samples['c'][idx]),
        'D_f': float(posterior_samples['D_f'][idx])
    })
    G_t_post = model_fit.predict(rheo_data, gamma_0=gamma_0, f_init=f_init)
    ax.plot(t_relax, G_t_post, 'r-', alpha=0.05, linewidth=1)

# Overlay data
ax.plot(t_relax, G_t_noisy, 'bo', markersize=5, alpha=0.6, label='Data', zorder=10)
ax.plot(t_relax, G_t_true, 'g--', linewidth=2, label='True model', zorder=11)

ax.set_xlabel('Time (s)')
ax.set_ylabel('G(t) (Pa)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Posterior Predictive Check (100 draws)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'posterior_predictive.png', dpi=300, bbox_inches='tight')
plt.show()

---

## 6. Fluidity Profile Homogenization During Relaxation

Visualize how spatial fluidity gradients diffuse over time.

In [ ]:
# Use median posterior parameters for simulation
median_params = {
    'G': float(np.median(posterior_samples['G'])),
    'eta_s': float(np.median(posterior_samples['eta_s'])),
    'lambda_0': float(np.median(posterior_samples['lambda_0'])),
    'tau_eq': float(np.median(posterior_samples['tau_eq'])),
    'a': float(np.median(posterior_samples['a'])),
    'c': float(np.median(posterior_samples['c'])),
    'D_f': float(np.median(posterior_samples['D_f']))
}

model_median = FluidityNonlocal(
    n_points=51,
    gap_width=params['gap_width'],
    coupling='full'
)
model_median.parameters.set_values(median_params)

# Simulate relaxation with median parameters
result_median = model_median.simulate_relaxation(
    gamma_0=gamma_0,
    t_end=t_end,
    n_times=n_times,
    f_init=f_init
)

f_profile_median = result_median['fluidity_profile']

In [ ]:
# Plot fluidity evolution: 2D heatmap + line profiles
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Heatmap (fluidity vs position and time)
ax = axes[0]
y_mm = y_grid * 1e3
t_plot, y_plot = np.meshgrid(t_relax, y_mm)

contour = ax.contourf(t_plot, y_plot, f_profile_median.T, levels=20, cmap='viridis')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Position y (mm)')
ax.set_xscale('log')
ax.set_title('Fluidity f(y, t) Evolution')
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('Fluidity f')

# Plot 2: Line profiles at specific times
ax = axes[1]
time_indices = [0, len(t_relax)//4, len(t_relax)//2, 3*len(t_relax)//4, -1]
colors = plt.cm.plasma(np.linspace(0, 1, len(time_indices)))

for i, idx in enumerate(time_indices):
    ax.plot(y_mm, f_profile_median[idx, :], color=colors[i], 
            linewidth=2, marker='o', markersize=4, label=f't={t_relax[idx]:.2e}s')

ax.set_xlabel('Position y (mm)')
ax.set_ylabel('Fluidity f(y)')
ax.set_title('Fluidity Profiles at Different Times')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'fluidity_homogenization.png', dpi=300, bbox_inches='tight')
plt.show()

# Quantify homogenization
f_variance = np.var(f_profile_median, axis=1)  # Variance across gap at each time

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t_relax, f_variance, 'b-', linewidth=2)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Fluidity Variance Var[f(y)]')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Spatial Homogenization Rate')
ax.grid(True, alpha=0.3)

# Estimate homogenization time (when variance drops to 1% of initial)
threshold = 0.01 * f_variance[0]
homogenization_idx = np.where(f_variance < threshold)[0]
if len(homogenization_idx) > 0:
    t_homog = t_relax[homogenization_idx[0]]
    ax.axvline(t_homog, color='r', linestyle='--', linewidth=2, 
               label=f'Homogenization time ≈ {t_homog:.2f}s')
    ax.legend()
    print(f"\nHomogenization time (Var[f] < 1% initial): {t_homog:.2f}s")
    print(f"Diffusive length scale: √(D_f·t_homog) = {np.sqrt(median_params['D_f']*t_homog)*1e3:.4f} mm")
    print(f"Gap width: {params['gap_width']*1e3:.4f} mm")

plt.tight_layout()
plt.savefig(output_dir / 'homogenization_rate.png', dpi=300, bbox_inches='tight')
plt.show()

---

## 7. Save Results

In [ ]:
# Save fitted parameters
np.savez(
    output_dir / 'fitted_params.npz',
    **median_params,
    gap_width=params['gap_width'],
    gamma_0=gamma_0
)

# Save posterior samples
np.savez(
    output_dir / 'posterior_samples.npz',
    **posterior_samples
)

# Save synthetic data
np.savez(
    output_dir / 'synthetic_data.npz',
    t=t_relax,
    G_t_true=G_t_true,
    G_t_noisy=G_t_noisy,
    f_profile=f_profile,
    f_init=f_init,
    y_grid=y_grid
)

# Save summary statistics
summary.to_csv(output_dir / 'posterior_summary.csv')

# Save ArviZ InferenceData
idata.to_netcdf(output_dir / 'inference_data.nc')

print(f"Results saved to {output_dir}/")
print(f"  - fitted_params.npz")
print(f"  - posterior_samples.npz")
print(f"  - synthetic_data.npz")
print(f"  - posterior_summary.csv")
print(f"  - inference_data.nc")
print(f"  - *.png (plots)")

---

## Key Takeaways

### 1. Spatial Relaxation Dynamics
- **Initial heterogeneity**: Fluidity profile f(y, t=0) can be non-uniform from prior shear history
- **Diffusive homogenization**: D_f controls spatial smoothing timescale τ_diff ~ h²/D_f
- **Structural aging**: τ_eq drives recovery toward equilibrium f → 1

### 2. Relaxation Modulus Decay
- **Elastic jump**: G(0⁺) = G (instantaneous)
- **Multi-timescale decay**: Controlled by λ_avg(t) = 1/⟨f(y,t)⟩ and τ_eq
- **Spatial coupling**: Nonlocal diffusion creates slower relaxation than local model

### 3. Parameter Identifiability
- **G**: Well-identified from G(0⁺)
- **D_f**: Controls homogenization rate (measurable from initial heterogeneity)
- **τ_eq, λ_0**: Control long-time decay
- **a, c**: Weak influence in relaxation (better identified in flow protocols)

### 4. NLSQ + Bayesian Workflow
- **NLSQ**: Fast point estimate (~seconds to minutes)
- **Bayesian**: Uncertainty quantification (~minutes with 4 chains)
- **Convergence**: R-hat < 1.01, ESS > 400 confirms reliable posteriors

### 5. Practical Insights
- **Homogenization time**: Critical for experimental design (wait time between tests)
- **Gap width dependence**: Larger gaps require longer relaxation for homogenization
- **Initial condition sensitivity**: Relaxation protocol reveals spatial effects better than steady shear

---

## Next Steps

1. **Compare to local model**: Quantify impact of spatial coupling
2. **Vary gap width**: Study h-dependence of relaxation timescales
3. **Different initial conditions**: Test response to various f(y, t=0) profiles
4. **Multi-protocol fitting**: Combine relaxation + startup + LAOS for better parameter constraints

---

**References:**
- Bocquet, L., Colin, A., & Ajdari, A. (2009). *Phys. Rev. Lett.* 103, 036001.
- Picard, G., Ajdari, A., Bocquet, L., & Lequeux, F. (2002). *Eur. Phys. J. E* 15, 371.
- Moorcroft, R. L., & Fielding, S. M. (2013). *Phys. Rev. Lett.* 110, 086001.